In [1]:
import os
from pathlib import Path
import sys

if "__project_dir__" not in globals():
    __project_dir__ = Path.cwd().parents[1].resolve()

sys.path.append(__project_dir__)
os.chdir(__project_dir__)

### Load data

In [11]:
import pandas as pd

In [12]:
df = pd.read_csv("/home/yutanagano/UCLOneDrive/MBPhD/projects/tcr_embedder/data/preprocessed/tanno/test.csv")

In [13]:
df

,TRAV,CDR3A,TRAJ,TRBV,CDR3B,TRBJ,Epitope,MHCA,MHCB,duplicate_count
0,TRAV13-1,CAASILDNDMRF,TRAJ43,TRBV15,CATSDWRPQGEETQYF,TRBJ2-5,NaN,NaN,NaN,NaN
1,TRAV12-3,CAMITNFGNEKLTF,TRAJ48,TRBV29-1,CSAAGTGSSYEQYF,TRBJ2-7,NaN,NaN,NaN,NaN
2,TRAV39,CAVEEGGADGLTF,TRAJ45,TRBV19,CASSLSPNYGYTF,TRBJ1-2,NaN,NaN,NaN,NaN
3,TRAV8-6,CAVSDEDYGGATNKLIF,TRAJ32,TRBV6-1,CASSGQGGTEAFF,TRBJ1-1,NaN,NaN,NaN,NaN
4,TRAV9-2,CALSGGSQGNLIF,TRAJ42,TRBV4-3,CASSQEARGENIQYF,TRBJ2-4,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
81844,TRAV29/DV5,CAAWTGTASKLTF,TRAJ44,TRBV20-1,CSARDLAVSYNEQFF,TRBJ2-1,NaN,NaN,NaN,NaN
81845,TRAV13-1,CAAGNNFNKFYF,TRAJ21,TRBV27,CASSTRGRGYEKLFF,TRBJ1-4,NaN,NaN,NaN,NaN
81846,TRAV8-4,CAVSEAPGYSTLTF,TRAJ11,TRBV4-1,CASSQGGGTGNTIYF,TRBJ1-3,NaN,NaN,NaN,NaN
81847,TRAV26-2,CILSYNFNKFYF,TRAJ21,TRBV29-1,CSVDGNTEAFF,TRBJ1-1,NaN,NaN,NaN,NaN


### OLGA setup

In [14]:
from olga import load_model
import olga.generation_probability as pgen

OLGA_PATH = Path(".venv") / "lib64" / "python3.8" / "site-packages" / "olga"
DEFAULT_HUMAN_T_BETA_PATH = OLGA_PATH / "default_models" / "human_T_beta"

In [6]:
params_file_name = DEFAULT_HUMAN_T_BETA_PATH / "model_params.txt"
marginals_file_name = DEFAULT_HUMAN_T_BETA_PATH / "model_marginals.txt"
v_anchor_pos_file = DEFAULT_HUMAN_T_BETA_PATH / "V_gene_CDR3_anchors.csv"
j_anchor_pos_file = DEFAULT_HUMAN_T_BETA_PATH / "J_gene_CDR3_anchors.csv"

In [7]:
genomic_data = load_model.GenomicDataVDJ()
genomic_data.load_igor_genomic_data(params_file_name, v_anchor_pos_file, j_anchor_pos_file)

In [9]:
generative_model = load_model.GenerativeModelVDJ()
generative_model.load_and_process_igor_model(marginals_file_name)

In [10]:
pgen_model = pgen.GenerationProbabilityVDJ(generative_model, genomic_data)

### Compute pGens

In [17]:
pgen_model.compute_aa_CDR3_pgen("CASSRRGKWNGYTF", "TRBV7-2", "TRBJ1-2")

1.7682625631576856e-12